# Topic Modelling

In this part of the assignment you will analyze a data set consisting of abstracts from the 2017 Conference on Neural Information Processing Systems (NeurIPS), The file `papers2017.csv` contans a list of 679 titles and abstracts from the conference proceedings. 

The task is to compute the posterior distribution in the LDA model that was discussed during the lectures. Set the hyperparameters of the prior to $\alpha = \eta = 1$. The number of topics can be set to $K = 5$.

Before we can begin with the inference we need to load and pre-process the data.

In [13]:
import pandas as pd
import gensim
import nltk
import numpy as np
import scipy
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
stemmer = SnowballStemmer('english')
nltk.download('wordnet')

documents = pd.read_csv('papers2017.csv')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Arturas.Aleksandraus\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package wordnet is already up-to-date!


# Pre-process the data

* **Tokenization:** Split each headline into words, lowercase the words and remove punctuation.
* **Small words:** Remove all words with less than 3 characters.
* **Stopwords:** Remove all stopwords.
* **Lemmatize:** Change words in third person to first person and all verbs into present.
* **Stem:** Reduce the words to their root form.

In [14]:
# lemmatizing and stemming
def lem_stem(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text,pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in STOPWORDS and len(token) >= 3:
            result.append(lem_stem(token))
    return result

Next step is to process all of the abstract through this pre-process

In [15]:
processed_docs = documents['abstract'].map(preprocess)

Finally we create our dictionary by filtering out words with less than 10 appearances and all words which appears in more than half of the documents.

We then create a bag of words, where each processed document gets replaced by a list of words and the number of times they appear.

In [16]:
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=10, no_above=0.5)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Collapsed Gibbs

**Q1:**  For a sweep of the collapsed Gibbs sampler we need to sample from the distribution $p(\mathbf{z} \mid \mathbf{w}, \alpha, \eta)$. Where $\mathbf{z}$ is the topic per word for the documents.

Write out the full distribution and simplify the computations as much as possible. 

Also explain how to obtain the posterior distribution from the variables $\mathbf{\theta}$ (topic proportions for each document) and $\beta$ (word distributions for each topic), based on the output of the Gibbs sampler. That is how do we find the distribution of these quantities given the distribution of $\mathbf{z}$.

_hint: see the slides_


We can analytic compute $\theta$ and $\beta$ and only sample $z$. The resulting marginalization leads to a simplify objective function that looks s following:
$$
p(z_{d,n} = k \mid \mathbf{z}^{-(d,n)}, \mathbf{w}, \alpha, \eta) \propto p(z_{d,n} = k \mid \mathbf{z}^{-(d,n)}, \alpha) \cdot p(w_{d,n} \mid z_{d,n} = k, \mathbf{w}^{-(d,n)}, \mathbf{z}^{-(d,n)}, \eta) \propto \hat{\theta}_{d,k} \hat{\beta}_{k,v}
$$

$$
\hat{\theta}_{d,k} = \frac{\alpha + c_{d,k}}{\sum_{k'=1}^{K}(\alpha + c_{d,k'})}
$$

$$
\hat{\beta}_{k,v} = \frac{\eta + \tilde{c}_{k,v}}{\sum_{v'=1}^{V}(\eta + \tilde{c}_{k,v'})}
$$

**Q2:** Implement the collapsed Gibbs sampler on the provided NeurIPS dataset to approximate the posterior $p(\mathbf{z} \mid \mathbf{w}, \alpha, \eta)$. To help with this you have code providing the outer loop, you need to implement the initialization algorithm and the function for calculating the probabilities of the posterior distribution.

Where `c` and `ct` are $c$ and $\tilde{c}$ from the slides. The count of each topic per document and the count of each topic per word.

In [17]:
# Initialization function, sets the topic for each word in each document at random.
def initialization_gibbs(bow_corpus, num_topics, num_words):
    num_docs = len(bow_corpus)
    z = []
    c = np.zeros((num_docs,num_topics), dtype = int) # c[d][k] = number of words in document d assigned to topic k
    ct = np.zeros((num_topics,num_words), dtype = int)
    for d in range(num_docs):
        topics_in_doc = np.zeros(0,dtype = int)
        for (v,i) in bow_corpus[d]:
            # Inner loop to set topic for each copy of a word independently.
            for _ in range(i):
                # Sample a random topic
                k = np.random.randint(num_topics) # how do we sample from categorical distribution? Uniform?
                # Update list of topics in document
                topics_in_doc = np.append(topics_in_doc,k)

                # Update c and ct based on document d, word v and topic k
                c[d][k] += 1
                ct[k][v] += 1
        z.append(topics_in_doc)
    return(z, c, ct)
# Calculates the probability for each topic on the current word and document.
def calc_probs(c, ct, d, v, alpha, eta, num_topics):
    p = np.zeros(num_topics)

    for k in range(num_topics):
        # For each topic calculate the probability that word v in document d belongs to topic k.
        theta = (c[d][k] + alpha)/ (c[d].sum() * alpha) 
        beta = (ct[k][v] + eta) / (ct[k].sum() * eta)
        p[k] = theta * beta
    # Normalize the vector before returning it.
    return p/sum(p)

In [18]:
# Hyperparameters
alpha = 0.1
eta = 0.1

# Number of iterations (number of samples from the posterior)
max_itr_gibbs = 10

# Number of topics in the model
num_topics = 10

# Number of words and documents (may help you later)
num_words = len(dictionary)
num_docs = len(bow_corpus)

# Start by initializing all values: z (topics) should be set randomly and c and ct (tilde c) should be calculated based on the randomly set topics.
(z, c, ct) = initialization_gibbs(bow_corpus, num_topics, num_words)

# Gibbs sampling:
for itr in range(max_itr_gibbs):
    for d in range(num_docs):
        # indx keeps track of the index of the words in each document
        indx = 0
        for (v,i) in bow_corpus[d]:
            for _ in range(i):
                # k is current topic
                k = z[d][indx]

                # Decrease c and ct based on the current topic
                c[d][k] -= 1
                ct[k][v] -= 1

                # Calculate probabilities for the posterior distribution
                probs = calc_probs(c, ct, d, v, alpha, eta, num_topics)
                

                # Sample new topic
                new_k = np.random.choice(num_topics, p = probs)

                # Increase c and ct based on the new topic
                c[d][new_k] += 1
                ct[new_k][v] += 1

                # Set the word (index indx) to the new topic
                z[d][indx] = new_k
                indx += 1

**Q3:** Present the top 5 words based on term-score for each topic, also give a name to each of the topics.

In [19]:
def beta(v, k, topic_word, eta):
    return (topic_word[k][v] + eta) / (topic_word[k].sum() + eta)

def term_score(v, k, topic_word, eta, num_topics):  
    bkv = beta(v, k, topic_word, eta)
    log_beta = np.log(bkv)
    log_sum_beta = sum([np.log(beta(v, t, topic_word, eta)) for t in range(num_topics)])
    return bkv*(log_beta - log_sum_beta/num_topics)

def rank_words_per_topic(topic_word, eta, num_topics, num_words, top = 5):
    score_word = []
    for k in range(num_topics):
        topic_score = []
        for v in range(num_words):
            topic_score.append((term_score(v, k, topic_word, eta, num_topics), v))
        topic_score.sort(reverse = True)
        score_word.append(topic_score)
    word = [[word for score, word in topic][:top] for topic in score_word]
    return word

for topic, words in enumerate(rank_words_per_topic(ct, eta, num_topics, num_words)):
    print(f'topic {topic}: ' + ', '.join([dictionary[id] for id in words]))

topic 0: converg, algorithm, sampl, distribut, regret
topic 1: network, architectur, train, recurr, neural
topic 2: train, network, neural, task, layer
topic 3: label, function, estim, polici, valu
topic 4: algorithm, gradient, function, descent, stochast
topic 5: imag, infer, agent, featur, dynam
topic 6: data, point, process, generat, attent
topic 7: imag, train, generat, gin, distribut
topic 8: graph, algorithm, causal, problem, queri
topic 9: cluster, algorithm, matrix, kernel, plan


# Variational inference

**Q4:** Write down explicit update expressions for a CAVI algorithm for approximating the posterior distribution $p(\mathbf{\theta},\mathbf{z},\beta \mid \mathbf{w}, \alpha, \eta)$.

$$
\phi_{d,n}^k \propto exp(\psi(\gamma_{d,k}) + \psi(\lambda_{k,{w_{d,n}}}) - \psi(\sum_v \lambda_{k,v})))
$$
$$
\gamma_{d} = \alpha + \sum_n \phi_{d,n}\\
\lambda_{k} = \eta + \sum_d \sum_n \mathbb{1}\{w_{d,n}=v\}\phi_{d,n}^k
$$

**Q5:** Implement the CAVI algorithm for the NeurIPS data. To help you with this task some code is provided that you need to fill in.

For the code we work with $\log(\phi)$, this simplifies some of the expressions and is in general more numericly stable.

In [20]:
# Initilization function, sets lambdas and gammas randomly, sets all phis to 0.
def initialization_cavi(bow_corpus, num_topics, num_words):
    num_docs = len(bow_corpus)
    logphis = []
    wordMatrix = []
    docLengths = []
    lambdas = np.random.gamma(1, size = (num_topics, num_words))
    gammas = np.random.gamma(1, size = (num_docs, num_topics))
    for d in range(num_docs):
        words_in_doc = 0
        word_vec = np.zeros(0, dtype = int)
        for (v,i) in bow_corpus[d]:
            words_in_doc += i
            for _ in range(i):
                word_vec = np.append(word_vec, v)
        logphis.append(np.zeros((words_in_doc,num_topics)))
        docLengths.append(words_in_doc)
        wordMatrix.append(word_vec)
    return(logphis, lambdas, gammas, docLengths, wordMatrix)


In [21]:
# Hyperparameters
alpha = 0.1
eta = 0.1

# Number of iterations (number of samples from the posterior)
max_itr_cavi = 10

# Number of topics in the model
num_topics = 10

# Number of words and documents (may help you later)
num_words = len(dictionary)
num_docs = len(bow_corpus)

# Start by initializing all values, we set all phis to zero and randomize lambdas and gammas from the gamma distribution. Also calculates doc_lengths and word_matrix.
(logphis, lambdas, gammas, doc_lengths, word_matrix) = initialization_cavi(bow_corpus, num_topics, num_words)

for itr in range(max_itr_cavi):
    for d in range(num_docs):
        indx = 0
        for (v,i) in bow_corpus[d]:
            for _ in range(i):
                for k in range(num_topics):
                    # Calculate each logphi based on the expected value of the natural parametrization.
                    # The digamma function is available in scipy.specieal.digamma
                    logphis[d][indx][k] = scipy.special.digamma(gammas[d, k]) + scipy.special.digamma(lambdas[k, v]) - scipy.special.digamma(lambdas[k, :].sum())
                # Normalize the logphis                                                                                 
                logphis[d][indx, :] = logphis[d][indx, :] - np.log(np.exp(logphis[d][indx, :]).sum())
                indx += 1
        for k in range(num_topics):
            # Calculate the gammas based on the phis
            gammas[d][k] = alpha + np.exp(logphis[d][:, k]).sum()
    for k in range(num_topics):
        for v in range(num_words):
            # Update the lambdas. 
            lambdas[k][v] = eta + np.sum([np.exp(logphis[d][word_matrix[d] == v, k]).sum() for d in range(num_docs)])

**Q6:** Present the top-5 words based on term-score for each topic and also give a name to each of the topics.

In [23]:
for topic, words in enumerate(rank_words_per_topic(lambdas, eta, num_topics, num_words)):
    print(f'topic {topic}: ' + ', '.join([dictionary[id] for id in words]))

topic 0: approxim, approach, problem, imag, matrix
topic 1: estim, predict, label, loss, represent
topic 2: train, object, time, bayesian, attent
topic 3: data, algorithm, regret, method, structur
topic 4: algorithm, cluster, method, sampl, time
topic 5: network, task, neural, inform, polici
topic 6: perform, time, generat, convex, approach
topic 7: problem, algorithm, task, likelihood, perform
topic 8: network, neural, base, covari, design
topic 9: method, estim, problem, data, paper


# Comparing Gibbs and Variational Inference

**Q7:** Choose one of the abstracts, present the top 5 topics of the document and present the title of the 5 closest other abstracts. Do this for both of the algorithms on the same abstract. Discuss similairities and differences between the results from the two algorithms.

In [26]:
def topics_for_doc(doc_id, doc_topics, topic_word, eta, num_topics, num_words, top = 5):
    topic_dist = doc_topics[doc_id]/doc_topics[doc_id].sum()
    # Get topical words for each topic
    topic_words = rank_words_per_topic(topic_word, eta, num_topics, num_words, top)
    # Sort the topics based on the topic alignment
    topic_dist_words = sorted(zip(topic_dist, range(num_topics), topic_words), reverse = True)
    return topic_dist_words[:top]

def document_similarity(doc_id, doc_topics, top = 5):
    theta = lambda c, d, k : (c[d][k] + alpha)/ (c[d].sum() * alpha) 
    # for every value  doc_topics apply theta function 
    thetas = np.zeros_like(doc_topics)
    for d in range(num_docs):
        for k in range(num_topics):
            thetas[d][k] = theta(doc_topics, d, k)
    thetas = np.sqrt(thetas)
    thetas -= thetas[doc_id]
    thetas = thetas**2
    similarity = thetas.sum(axis = 1)
    # similarity[doc_id] = np.inf # Set the similarity to itself to infinity so its not returned
    similarity_index = np.argsort(similarity)
    return list(zip(similarity[similarity_index][:top], similarity_index[:top]))
    

document = 38
title = documents.iloc[document]['title']
abstract = documents.iloc[document]['abstract']

print(f'Title: {title}')
print(f'Abstract: {abstract}')

print('Gibbs Topics:')
for topic_score, topic_id, words in topics_for_doc(document, c, ct, eta, num_topics, num_words):
    print(f'topic {topic_id}, aliment: {topic_score:.2f}, ' + ', '.join([dictionary[id] for id in words]))
print('Gibbs Document Similarity:')
for similarity, doc_id in document_similarity(document, c):
    print(f'Doc {doc_id}, similarity: {similarity:.2f}, title: {documents.iloc[doc_id]["title"]}')
print()
print('CAVI Topics:')
for topic_score, topic_id, words in topics_for_doc(document, gammas, lambdas, eta, num_topics, num_words):
    print(f'topic {topic_id}, aliment: {topic_score:.2f}, ' + ', '.join([dictionary[id] for id in words]))
print('CAVI Document similarity:')
for similarity, doc_id in document_similarity(document, gammas):
    print(f'Doc {doc_id}, similarity: {similarity:.2f}, title: {documents.iloc[doc_id]["title"]}')


Title: Pose Guided Person Image Generation
Abstract: This paper proposes the novel Pose Guided Person Generation Network (PG$^2$) that allows to synthesize person images in arbitrary poses, based on an image of that person and a novel pose. Our generation framework PG$^2$ utilizes the pose information explicitly and consists of two key stages: pose integration and image refinement. In the first stage the condition image and the target pose are fed into a U-Net-like network to generate an initial but coarse image of the person with the target pose. The second stage then refines the initial and blurry result by training a U-Net-like generator in an adversarial way. Extensive experimental results on both 128$\times$64 re-identification images and 256$\times$256 fashion photos show that our model generates high-quality person images with convincing details.
Gibbs Topics:
topic 7, aliment: 0.42, imag, train, generat, gin, distribut
topic 5, aliment: 0.31, imag, infer, agent, featur, dynam
t

**Q8:** Discuss the key conceptual differences between the Gibbs sampler and the CAVI algorithm. What are the pros and cons of each method? Which method do you prefer and why?